In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [39]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('bought_insurance', axis='columns'), df['bought_insurance'], random_state=37)

X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [41]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

input_layer = Input(shape=(2,))

dense_layer = Dense(1, activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')(input_layer)

model = Model(inputs=input_layer, outputs=dense_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step - accuracy: 0.4762 - loss: 0.7171
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4762 - loss: 0.7167
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4762 - loss: 0.7163
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4762 - loss: 0.7158
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4762 - loss: 0.7154
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4762 - loss: 0.7150
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4762 - loss: 0.7146
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4762 - loss: 0.7142
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4762 - loss: 0.7138
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4762 - loss: 0.7134
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.4762 - loss: 0.7130
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy

In [45]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8571 - loss: 0.4513


[0.4512661397457123, 0.8571428656578064]

In [46]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.8369946],
        [1.5844278]], dtype=float32),
 array([-2.9815736], dtype=float32))

In [75]:
def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred_new = [max(i, epsilon) for i in y_pred]
    y_pred_new = [min(i, 1-epsilon) for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    return -np.mean(y_true * np.log(y_pred_new) + (1 - y_true) * np.log(1 - y_pred_new))

In [76]:
def sigmoid_numpy(X):
    return 1 / (1 + np.exp(-X))

sigmoid_numpy(np.array([1, 2, 3]))

array([0.73105858, 0.88079708, 0.95257413])

In [77]:
def grad_descent(age, aff, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.01
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * aff + bias
        y_pred = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_pred)
        w1d = (1 / n) * np.dot(np.transpose(age), (y_pred - y_true))
        w2d = (1 / n) * np.dot(np.transpose(aff), (y_pred - y_true))
        biasd = np.mean(y_pred - y_true)

        w1 = w1 - w1d * rate
        w2 = w2 - w2d * rate
        bias = bias - biasd * rate

        print(f'Epoch: {i}; w1: {w1}; w2: {w2}; bias: {bias}; loss: {loss}')

        if loss <= loss_threshold:
            break

    return w1, w2, bias

In [81]:
grad_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 20000, 0.4413)

Epoch: 0; w1: 0.9994615400145692; w2: 0.9989219414958425; bias: -0.002507860273647024; loss: 0.7170837780331669
Epoch: 1; w1: 0.9989253486791114; w2: 0.997847863641895; bias: -0.0050095709786066546; loss: 0.7163106732381802
Epoch: 2; w1: 0.9983914237700267; w2: 0.9967777640264408; bias: -0.007505139242605441; loss: 0.7155417455858039
Epoch: 3; w1: 0.9978597630337802; w2: 0.9957116401718995; bias: -0.009994572266349033; loss: 0.7147769789983078
Epoch: 4; w1: 0.9973303641870852; w2: 0.9946494895351509; bias: -0.01247787732300886; loss: 0.7140163573796826
Epoch: 5; w1: 0.9968032249170868; w2: 0.993591309507861; bias: -0.014955061757706379; loss: 0.7132598646165634
Epoch: 6; w1: 0.9962783428815478; w2: 0.992537097416812; bias: -0.017426132986995; loss: 0.7125074845791537
Epoch: 7; w1: 0.9957557157090347; w2: 0.991486850524235; bias: -0.0198910984983397; loss: 0.7117592011221414
Epoch: 8; w1: 0.9952353409991052; w2: 0.9904405660281468; bias: -0.02234996584959446; loss: 0.7110149980856129
Ep

(4.7562699043846015, 1.7627194012472727, -3.1204781512252184)

In [82]:
coef, intercept

(array([[4.8369946],
        [1.5844278]], dtype=float32),
 array([-2.9815736], dtype=float32))